In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.optimize import minimize

from aeons.covariance import logX_mu, logX_covinv_chol, points_at_iteration, logPr

# Load test samples
from aeons.tools import pickle_in
samples_g_1, samples_c_1, samples_w_1 = pickle_in("../test_samples/samples_1.pickle")

from aeons.true_distribution import generate_Xs, generate_Xsamples

In [2]:
def mean_X(nk):
    return np.cumprod(nk/(nk+1))

def cov_X(nk):
    t_1 = np.cumprod(nk/(nk+1)) # cumulative product of expectation of t
    t_2 = np.cumprod(nk/(nk+2)) # cumulative product of expectation of t^2
    iterations = len(nk)
    cov_X = np.zeros((iterations, iterations))
    for i in range(iterations):
        cov_X[i][i] = t_2[i] - t_1[i]**2 
        for j in range(i+1, iterations): # start j above i so min(i,j) automatically fulfilled
            correlated = t_2[i] - t_1[i]**2
            independent = t_1[j]/t_1[i] # cumulative product from i+1 to j
            cov_X[i][j] = cov_X[j][i] = correlated * independent
    return cov_X

In [32]:
def minimise_theta(inverse, logL, mean, cov_inv, x0):
    def func(theta):
        X = inverse(logL, theta)
        return (X - mean).T @ cov_inv @ (X - mean) # want to maximise probability <-> minimise negative
    solution = minimize(func, x0)
    return solution

def minimise_model(func, inverse, nk, mean, covinv, x0, dtrue=10):
    Xs = generate_Xs(nk)
    y = func(Xs, dtrue)
    dbest = minimise_theta(inverse, y, mean, covinv, x0).x
    return dbest

In [62]:
def linear(X, theta):
    return theta * X

def linear_inverse(y, theta):
    return y/theta

def quad(X, theta):
    return theta * X**2

def quad_inverse(y, theta):
    return (y/theta)**(1/2)

def log(X, theta):
    return theta * np.log(X)

def log_inverse(y, theta):
    return np.exp(y/theta)

In [57]:
iteration = 100
nk = 1000 * np.ones(iteration)
Xmean = mean_X(nk)
Xcovinv = np.linalg.inv(cov_X(nk))

In [61]:
repeats = 100
cs = np.zeros(repeats)
ls = np.zeros(repeats)
for i in range(repeats):
    cs[i] = minimise_model(linear, linear_inverse, nk, Xmean, Xcovinv, 10)
    ls[i] = minimise_model(linear, linear_inverse, nk, Xmean, np.eye(len(Xmean)), 10)

print(cs.mean(), cs.std())
print(ls.mean(), ls.std())

10.000716104743917 0.010232970091874065
10.000342579306022 0.06062351415398248


In [60]:
repeats = 100
cs = np.zeros(repeats)
ls = np.zeros(repeats)
for i in range(repeats):
    cs[i] = minimise_model(quad, quad_inverse, nk, Xmean, Xcovinv, 10)
    ls[i] = minimise_model(quad, quad_inverse, nk, Xmean, np.eye(len(Xmean)), 10)

print(cs.mean(), cs.std())
print(ls.mean(), ls.std())

9.996928709377865 0.027062338854309406
10.009659011927058 0.11709087047536451


In [63]:
repeats = 100
cs = np.zeros(repeats)
ls = np.zeros(repeats)
for i in range(repeats):
    cs[i] = minimise_model(log, log_inverse, nk, Xmean, Xcovinv, 10)
    ls[i] = minimise_model(log, log_inverse, nk, Xmean, np.eye(len(Xmean)), 10)

print(cs.mean(), cs.std())
print(ls.mean(), ls.std())

19.573750569482264 2.5043261689101315
10.011469704436289 1.210932094398764
